# 12wk-1: ???

최규빈  
2023-11-20

<a href="https://colab.research.google.com/github/guebin/DV2023/blob/main/posts/12wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-wFUcEr7JtSb7MWUYl6QrW_&si=CmWA765p0mnwWTGA>

# 2. Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import requests
import json 

In [2]:
pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

In [3]:
us_dict = json.loads(requests.get('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json').text)

In [4]:
newyork_dict = us_dict.copy()
newyork_dict['features'] = [l for l in us_dict['features'] if "New York" in l['properties']['NAME']]

# 3.

In [5]:
df = pd.read_csv("NYCTaxi.csv")[::100].reset_index(drop=True)
df.columns

In [6]:
df

In [7]:
fig = px.scatter_mapbox(
    data_frame=df,
    lat='pickup_latitude',
    lon='pickup_longitude',
    opacity=0.3,
    center={'lat': 40.7322, 'lon': -73.9052},
    mapbox_style='carto-positron',
    zoom=10,
    width=800,
    height=600
)
fig.update_traces(
    marker={'size':2}
)
fig.show(config={'scrollZoom': False})

In [8]:
fig = px.density_mapbox(
    data_frame=df,
    lat = 'pickup_latitude',
    lon = 'pickup_longitude',
    center = {'lat':40.7322, 'lon':-73.9052},
    radius = 1,
    #---#
    mapbox_style='carto-positron',
    zoom=10,
    width=800,
    height=600
)
fig.show(config={'scrollZoom': False})

In [9]:
fig = px.density_mapbox(
    data_frame=df,
    lat = 'pickup_latitude',
    lon = 'pickup_longitude',
    center = {'lat':40.7322, 'lon':-73.9052},
    radius = 2,
    z='passenger_count',
    #---#
    mapbox_style='carto-positron',
    zoom=10,
    width=800,
    height=600
)
fig.show(config={'scrollZoom': False})

In [10]:
df.assign(log_trip_duration = lambda df: np.log(df.trip_duration))

In [11]:
fig = px.density_mapbox(
    data_frame=df.assign(log_trip_duration = lambda df: np.log(df.trip_duration)),
    lat = 'pickup_latitude',
    lon = 'pickup_longitude',
    center = {'lat':40.7322, 'lon':-73.9052},
    radius = 1.5,
    z='log_trip_duration',
    #---#
    mapbox_style='carto-positron',
    zoom=10,
    width=800,
    height=600
)
fig.show(config={'scrollZoom': False})

In [12]:
df.assign(alone = df.passenger_count == 1).assign(vendor_id = lambda df: df.vendor_id.astype(str))

In [13]:
fig = px.scatter_mapbox(
    data_frame=df.assign(alone = df.passenger_count == 1).assign(vendor_id = lambda df: df.vendor_id.astype(str)),
    lat = 'pickup_latitude',
    lon = 'pickup_longitude',
    opacity = 0.3,
    center = {'lat':40.7322, 'lon':-73.9052},
    color = 'vendor_id',
    #---#
    mapbox_style='carto-positron',
    zoom=10,
    width=800,
    height=600
)
fig.update_traces(
    marker={'size':2}
)
fig.show(config={'scrollZoom': False})

In [14]:
# df.pickup_datetime.str.split(' ').str[-1].str.split(':').str[0].apply(int)
# df.pickup_datetime.apply(pd.to_datetime).dt.hour

In [15]:
tidydata = df.assign(
    alone = df.passenger_count == 1,
    hour = df.pickup_datetime.apply(pd.to_datetime).dt.hour,
    vendor_id = df.vendor_id.astype(str),
    log_trip_duration = np.log(df.trip_duration)>8
).sort_values(by='hour')

In [16]:
fig = px.scatter_mapbox(
    data_frame=tidydata,
    lat = 'pickup_latitude',
    lon = 'pickup_longitude',
    center = {'lat':40.7322, 'lon':-73.9052},
    color = 'vendor_id',
    size = 'passenger_count',
    size_max = 5,
    animation_frame = 'hour',
    #---#
    mapbox_style='carto-positron',
    zoom=10,
    width=800,
    height=600
)
fig.show(config={'scrollZoom': False})

In [17]:
tidydata\
.groupby('vendor_id').agg({'passenger_count':'mean'})\
.reset_index()\
.plot.bar(x='vendor_id',y='passenger_count',color='vendor_id',text='passenger_count')

In [18]:
tidydata.log_trip_duration.min(), tidydata.log_trip_duration.max()

In [19]:
fig = px.scatter_mapbox(
    data_frame=tidydata,
    lat = 'pickup_latitude',
    lon = 'pickup_longitude',
    center = {'lat':40.7322, 'lon':-73.9052},
    color = 'log_trip_duration',
    size = 'passenger_count',    
    size_max = 5,
    animation_frame = 'hour',
    range_color = (tidydata.log_trip_duration.min(), tidydata.log_trip_duration.max()),
    #---#
    mapbox_style='carto-positron',
    zoom=10,
    width=800,
    height=600
)
fig.show(config={'scrollZoom': False})

In [20]:
df_small = df[::100].reset_index(drop=True)

In [21]:
df_small

In [22]:
def transform(df):
    pick_up = df.loc[:,['id','pickup_datetime','pickup_longitude','pickup_latitude']].set_axis(['id','datetime','lon','lat'],axis=1).eval('state = "pickup"')
    drop_off = df.loc[:,['id','dropoff_datetime','dropoff_longitude','dropoff_latitude']].set_axis(['id','datetime','lon','lat'],axis=1).eval('state = "dropoff"')
    return pd.concat([pick_up,drop_off],axis=0).reset_index(drop=True)

In [23]:
pd.concat([transform(df) for i,df in df_small.groupby('id')]).reset_index(drop=True)\
.merge(df_small.drop(['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_datetime','dropoff_longitude','dropoff_latitude'],axis=1))

In [24]:
df2 = pd.concat([transform(df) for i,df in df_small.groupby('id')]).reset_index(drop=True)\
.merge(df_small.drop(['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_datetime','dropoff_longitude','dropoff_latitude'],axis=1))

In [25]:
df2

In [26]:
df2.assign(
    alone = lambda df: df.passenger_count == 1,
    hour = lambda df: df.datetime.apply(pd.to_datetime).dt.hour
)

In [27]:
fig = px.line_mapbox(
    df2.assign(alone = lambda df: df.passenger_count == 1, hour = lambda df: df.datetime.apply(pd.to_datetime).dt.hour).sort_values('hour'), 
    lat="lat", 
    lon="lon", 
    center = {'lat':40.7322, 'lon':-73.9052},
    line_group = 'id',
    color='alone',
    hover_data = 'hour',
    #---#
    mapbox_style='carto-positron',
    zoom=10,
    width=800,
    height=600,
)
fig.add_trace(
    px.scatter_mapbox(
        data_frame=df2.assign(alone = lambda df: df.passenger_count == 1, hour = lambda df: df.datetime.apply(pd.to_datetime).dt.hour).sort_values('hour'),
        lat = 'lat',
        lon = 'lon',
        center = {'lat':40.7322, 'lon':-73.9052},
        color = 'alone',
        size = 'trip_duration',
        size_max = 10,
    ).data[0]
)
fig.add_trace(
    px.scatter_mapbox(
        data_frame=df2.assign(alone = lambda df: df.passenger_count == 1, hour = lambda df: df.datetime.apply(pd.to_datetime).dt.hour).sort_values('hour'),
        lat = 'lat',
        lon = 'lon',
        center = {'lat':40.7322, 'lon':-73.9052},
        color = 'alone',
        size = 'trip_duration',
        size_max = 10,
    ).data[1]
)
fig.update_traces(
    line={
        'width':1,
    },
    opacity=0.8
)
fig.show(config={'scrollZoom': False})